In [1]:
class Platform:
    WINDOWS = "windows"
    MACOS = "macos"

In [2]:
root_directory = "D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling"
platform = Platform.WINDOWS

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import sys
sys.path.append(root_directory)

import os
from os.path import join, getsize
import threading
import time
import subprocess
import re

import file_utils as fu

In [4]:
json_dir_path = fu.relative_to_absolute_path(
    "GetAllSongHTML/song_list_link_by_artist/json",
    root_path=root_directory
)

print(json_dir_path)

D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json


In [5]:
song_links_by_alphabet = fu.get_file_path_list_in_dir(json_dir_path)

song_links_by_alphabet = {
    path.split("/")[-1].split(".")[0].lower() : path
    for path in song_links_by_alphabet
}

print(song_links_by_alphabet)

{'a': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/a.json', 'b': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/b.json', 'c': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/c.json', 'd': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/d.json', 'e': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/e.json', 'f': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/f.json', 'g': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/g.json', 'h': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/h.json',

In [6]:
def song_link_to_relative_html_file_path(link):
    artist_letter = link.split("/")[-2][0]
    artist = link.split("/")[-2]
    song = link.split("/")[-1]

    return f"{artist_letter}/{artist}/{song}.html"


print(
    song_link_to_relative_html_file_path(
        fu.read_data_from_json_file(
            song_links_by_alphabet["a"])[0]["link"]
        )
    )

a/a-boogie-wit-da-hoodie/drowning.html


In [7]:
def scroll_to_bottom(browser, scroll_pause_time = 0.5):
    # Get the screen height of the web
    screen_height = browser.execute_script("return window.screen.height;") 
    i = 1

    max_scroll_times = 25

    while True:
        print(f"Scrolled: {i} time(s)")

        # Scroll one screen height each time
        try:
            browser.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        except:
            print(f"Error while scrolling")
            raise Exception("Error while scrolling")
        
        time.sleep(scroll_pause_time)
        
        # Cập nhật scroll_height sau mỗi lần scroll, vì scroll_height có thể thay đổi sau khi scroll trang
        scroll_height = browser.execute_script("return document.body.scrollHeight;")

        i += 1
        if (screen_height) * i > scroll_height:
            break

        if i > max_scroll_times:
            print(f"Max scroll times reached")
            raise Exception("Max scroll times reached")

In [8]:
def scroll_and_get_html_content(url, webdriver_instance):
    try:
        webdriver_instance.get(url)
    except:
        print(f"Failed to get {url}")
        raise Exception(f"Failed to get {url}")
    
    scroll_to_bottom(webdriver_instance)
    
    contents = webdriver_instance.page_source

    return contents

In [9]:
def get_song_raw_html(url, webdriver_instance):
    print(f"{url}: processing...")
    
    page_source = scroll_and_get_html_content(url, webdriver_instance)

    path_to_save_html = fu.relative_to_absolute_path(
        song_link_to_relative_html_file_path(url),
        root_path=fu.relative_to_absolute_path(
            "GetAllSongHTML/song_list/raw_html",
            root_path=root_directory
        )
    )
    
    os.makedirs(os.path.dirname(path_to_save_html), exist_ok=True)

    fu.write_data_to_html_file(page_source, path_to_save_html)

    print(f"{url}: done")

In [10]:
def open_edge_in_remote_debugging_mode(port, platform, browser_instance_data_dir):
    print(f"Opening Edge in remote debugging mode on port {port}...")

    macos_command = f'''open -na "Microsoft Edge.app" --args
                        --remote-debugging-port={port}
                        --user-data-dir="{browser_instance_data_dir}"'''

    windows_command = f'''start msedge.exe 
                        --remote-debugging-port={port} 
                        --user-data-dir="{browser_instance_data_dir}'''

    command = macos_command if platform == Platform.MACOS else windows_command
    
    os.system(command.replace("\n", " "))

In [11]:
def init_selenium_in_remote_debugging_mode(port):
    options = webdriver.EdgeOptions()
    options.add_experimental_option("debuggerAddress", f"localhost:{port}")

    browser = webdriver.Edge(options=options)
    # browser.maximize_window()

    return browser

In [12]:
def windows_get_PID_of_process_running_on_port(port):
    command = f"netstat -a -n -o | findstr :{port}"

    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    #   TCP    127.0.0.1:40000        0.0.0.0:0              LISTENING       6920
    regex_pattern = r"\s+TCP\s+127.0.0.1:\d+\s+\d+.\d+.\d+.\d+:\d+\s+LISTENING\s+(.+)"

    match = re.search(regex_pattern, result.stdout)

    pid = None

    if match:
        pid = match.group(1).replace(" ", "")
    else:
        pid = None

    return pid

In [13]:
def kill_process_running_on_port(port, platform=Platform.WINDOWS):
    print(f"Killing process running on port {port}...")

    try:
        if platform == Platform.MACOS:
            result = os.system(f"lsof -ti tcp:{port} | xargs kill -9")
        else:
            pid = windows_get_PID_of_process_running_on_port(port)

            if pid:
                result = subprocess.run(
                    f"taskkill /PID {pid} /F", 
                    shell=True, 
                    capture_output=True, 
                    text=True
                ).stdout
            else:
                result = f"No process is running on port {port}"
    except:
        print(f"Error when killing process running on port {port}")

In [14]:
class BrowserStatus:
    AVAILABLE = "available"
    BUSY = "busy"

In [15]:
webdriver_pool = []

def init_webdriver_pool(n_webdriver=5):
    for i in range(n_webdriver):
        port = 40000 + i

        kill_process_running_on_port(port, platform)

        open_edge_in_remote_debugging_mode(
            port,
            platform,
            fu.relative_to_absolute_path(
                f"GetAllSongHTML/song_list/browser_instance_data/{port}",
                root_path=root_directory
            )
        )

        browser = init_selenium_in_remote_debugging_mode(port)

        webdriver_pool.append(
            {
                "port": port,
                "browser": browser,
                "browser_status": BrowserStatus.AVAILABLE
            }
        )

In [16]:
def change_port_status(port, status):
    for i in range(len(webdriver_pool)):
        if webdriver_pool[i]["port"] == port:
            webdriver_pool[i]["browser_status"] = status
            break

In [17]:
def get_raw_html_with_webdriver_from_pool(url, webdriver_pool_item):
    for browser in webdriver_pool:
        if (browser["port"] == webdriver_pool_item["port"] 
            and browser["browser_status"] == BrowserStatus.AVAILABLE):
            change_port_status(browser["port"], BrowserStatus.BUSY)
            
            try:
                get_song_raw_html(url, webdriver_pool_item["browser"])
            except:
                print(f"Error when processing {url}")
                
            change_port_status(browser["port"], BrowserStatus.AVAILABLE)
            
            break

In [18]:
def get_song_raw_html_by_alphabet(letter, song_per_batch=5):
    # Lấy danh sách link của letter
    song_links = fu.read_data_from_json_file(song_links_by_alphabet[letter])

    # Xóa các song đã được crawl
    song_links = [song for song in song_links
        if not os.path.exists(
            fu.relative_to_absolute_path(
                song_link_to_relative_html_file_path(song["link"]),
                root_path=fu.relative_to_absolute_path(
                    "GetAllSongHTML/song_list/raw_html",
                    root_path=root_directory
                )
            )
        )
    ]

    while(len(song_links) > 0):
        current_song_per_batch = song_per_batch if song_per_batch <= len(song_links) else len(song_links)
        song_batch = [song_links.pop(0) for _ in range(current_song_per_batch)]

        threadBatch = []

        for webdriver_pool_item in webdriver_pool:
            if webdriver_pool_item["browser_status"] == BrowserStatus.AVAILABLE:
                threadBatch.insert(
                    0,
                    threading.Thread(
                        target=get_raw_html_with_webdriver_from_pool,
                        args=(song_batch.pop(0)["link"], webdriver_pool_item, )
                    )
                )

                threadBatch[0].start()
            elif webdriver_pool_item == webdriver_pool[-1]:
                print(f"All webdrivers are busy")
                break
        
        for i in range(len(threadBatch)):
            threadBatch[i].join()


        anti_rate_limiting_waited_time = 0
        total_time_to_wait = 60

        for i in range(total_time_to_wait):
            time.sleep(1)
            anti_rate_limiting_waited_time += 1
            print(f"Waited {anti_rate_limiting_waited_time} of {total_time_to_wait} seconds")
        
        threadBatch.clear()

In [19]:
song_per_batch = 5

init_webdriver_pool(n_webdriver=song_per_batch)

Killing process running on port 40000...
Opening Edge in remote debugging mode on port 40000...


Killing process running on port 40001...
Opening Edge in remote debugging mode on port 40001...
Killing process running on port 40002...
Opening Edge in remote debugging mode on port 40002...
Killing process running on port 40003...
Opening Edge in remote debugging mode on port 40003...
Killing process running on port 40004...
Opening Edge in remote debugging mode on port 40004...


In [20]:
get_song_raw_html_by_alphabet('a', song_per_batch)

https://www.hooktheory.com/theorytab/view/adam-routt/the-sounds-of-space: processing...
https://www.hooktheory.com/theorytab/view/adam-routt/zzz: processing...
https://www.hooktheory.com/theorytab/view/adam-sandler/somebody-kill-me: processing...
https://www.hooktheory.com/theorytab/view/adam-szabo-and-johan-vilborg/two-to-one: processing...
https://www.hooktheory.com/theorytab/view/adam-wright/high-demand: processing...
Scrolled: 1 time(s)
Scrolled: 1 time(s)
Scrolled: 1 time(s)
Scrolled: 1 time(s)
Scrolled: 2 time(s)
Scrolled: 1 time(s)
Scrolled: 2 time(s)
Scrolled: 3 time(s)
Scrolled: 2 time(s)
Scrolled: 2 time(s)
Scrolled: 3 time(s)
https://www.hooktheory.com/theorytab/view/adam-wright/high-demand: done
Scrolled: 2 time(s)
Scrolled: 3 time(s)
Scrolled: 3 time(s)
https://www.hooktheory.com/theorytab/view/adam-szabo-and-johan-vilborg/two-to-one: done
Scrolled: 4 time(s)
Scrolled: 3 time(s)
Scrolled: 5 time(s)
Scrolled: 4 time(s)
Scrolled: 5 time(s)
Scrolled: 6 time(s)
Scrolled: 4 tim

In [1]:
# Kill all Edge instances
for webdriver in webdriver_pool:
    kill_process_running_on_port(webdriver["port"], platform)

NameError: name 'webdriver_pool' is not defined